##Write a Python script to collect 5000 tweets from the Twitter AP on a topic of your choice.

In [ ]:
import json
import pandas as pd
import oauth2
import time
import urllib2

### We first call the Twitter API (WITHOUT using the utility in R) for our given set of search terms "Superbowl50", "SB50", "BroncosCountry", "PanthersPride"

In [ ]:
url1 = "https://api.twitter.com/1.1/search/tweets.json"
params = {
"oauth_version": "1.0",
"oauth_nonce": oauth2.generate_nonce(),
"oauth_timestamp": int(time.time())
}
consumer = oauth2.Consumer(key="Gp3VB79tQPhnu3cRkCn0IqjFu", secret="4Yr4uQGfFx5njHXlSLgb98D3Mw7HYmGrNkE94gVq9cq9Ykalb8")
token = oauth2.Token(key="3306479018-27SzIFeeZwKQml4aNxx4U4gKwNZt5M0gqBSyh5Z", secret="imVFr8TCljR5MxbQKsKIwoXAERr0yU2Ah8oyuR8UGQzpK")
params["Gp3VB79tQPhnu3cRkCn0IqjFu"] = consumer.key
params["4Yr4uQGfFx5njHXlSLgb98D3Mw7HYmGrNkE94gVq9cq9Ykalb8"] = token.key


prev_id = int("693801035224055808")
for i in range(1):
    url = url1
    params["q"] = ["Superbowl50", "SB50", "BroncosCountry", "PanthersPride"]
    params["count"] = 500
    params["geocode"] = ""
    params["lang"] = ""
    params["locale"] = ""
    params["result_type"] = "popular" # Example Values: mixed, recent, popular
    params["until"] = ""
    params["since_id"] = ""
    params["max_id"] = str(prev_id)
    req = oauth2.Request(method="GET", url=url, parameters=params)
    signature_method = oauth2.SignatureMethod_HMAC_SHA1()
    req.sign_request(signature_method, consumer, token)
    #headers = req.to_header()
    url = req.to_url()
    response = urllib2.Request(url)

    data = json.load(urllib2.urlopen(response))
    if data["statuses"] == []:
        print "end of data"
        break
    else:
        prev_id = int(data["statuses"][-1]["id"]) - 1
        print prev_id, i
    f = open("outfile_" + str(i) + ".json", "w")
    json.dump(data["statuses"], f)
    f.close()
    time.sleep(5)

### We then convert the JSON file to a Dataframe that can be used for further processing. We have picked up only the relevant features for subsequent analysis

In [ ]:
rawdata = []
# Fill in the path below with your JSON file
with open('outfile_0.json') as f:
    for line in f:
        rawdata.append(json.loads(line))
sbtweet = pd.DataFrame(rawdata)

In [24]:
import pandas as pd

sbtweet = pd.read_csv('Superbowl Tweet.csv')

sbtweet.head()

,Date,Tweet,User Handle,In Reply To,Follower,Following,Listed,Retweet
0,1/29/16 23:59,RT @Panthers: Always With Us. #TEAM \r\r#SB50 ...,chanchanmans,NaN,167,137,1,861
1,1/29/16 23:59,The #SuperBowl crew paints both end zones for ...,PatGeraghtyinIA,NaN,4,8,2,0
2,1/29/16 23:59,RT @LevisStadium: Alright @Panthers. Your end ...,3timeallstar99,NaN,191,446,18,65
3,1/29/16 23:59,RT @Panthers: Ready For Battle. \r\r#SB50 #Kee...,jessicavoncanno,NaN,1191,582,2,1574
4,1/29/16 23:59,RT @SapphireLV: #SuperBowl50 with @therealgian...,VEGASandTONIC,NaN,1635,244,260,18


### We then normalise and standardise the available metrics

In [25]:
cols_to_norm = ['Follower','Following','Listed','Retweet']
sbtweet[cols_to_norm] = sbtweet[cols_to_norm].apply(lambda x: (x - x.mean()) / (x.max() - x.min()))

sbtweet.head()

,Date,Tweet,User Handle,In Reply To,Follower,Following,Listed,Retweet
0,1/29/16 23:59,RT @Panthers: Always With Us. #TEAM \r\r#SB50 ...,chanchanmans,NaN,-0.003814,-0.009888,-0.007702,0.076648
1,1/29/16 23:59,The #SuperBowl crew paints both end zones for ...,PatGeraghtyinIA,NaN,-0.003907,-0.011234,-0.007628,-0.076392
2,1/29/16 23:59,RT @LevisStadium: Alright @Panthers. Your end ...,3timeallstar99,NaN,-0.003800,-0.006663,-0.006451,-0.064838
3,1/29/16 23:59,RT @Panthers: Ready For Battle. \r\r#SB50 #Kee...,jessicavoncanno,NaN,-0.003231,-0.005244,-0.007628,0.203381
4,1/29/16 23:59,RT @SapphireLV: #SuperBowl50 with @therealgian...,VEGASandTONIC,NaN,-0.002979,-0.008771,0.011353,-0.073192


### We then use the 'Mean Decrease in Gini Coefficient' metric from Assignment 1 to assign weights for "Followers", "Listed Count", and "Retweets"

We perform a softmax transformation on the metric to use the weights

In [26]:
import numpy as np
npa = np.array

def transform(w, t = 1.0):
    e = (npa(w) / t)
    dist = e / np.sum(e)
    return dist

if __name__ == '__main__':

    w = np.array([1.4729,1.407,1.2518]) 
    #These are the Mean Decrease in Gini for Followers, Listed Count, and RTs for weights
    print transform(w)
    
wt = transform(w)


[ 0.35648764  0.34053779  0.30297456]


We then discovered that the RT count attributed to a tweet corresponds to the count of the ORIGINAL tweet and not the act of retweeting itself.

At this stage, we know that a Retweet count should be contribute towards 'influence' only for the original tweeter and NOT the person who retweets it. Hence, we artifically set the Retweet counts of RTs to 0

In [58]:

sbtweet['subs'] = sbtweet.apply(lambda x: x['Tweet'][:2], axis=1)
sbtweet.loc[sbtweet['subs']=='RT', 'Retweet'] = 0

sbtweet.head(10)


,Date,Tweet,User Handle,In Reply To,Follower,Following,Listed,Retweet,subs
0,1/29/16 23:59,RT @Panthers: Always With Us. #TEAM \r\r#SB50 ...,chanchanmans,NaN,-0.003814,-0.009888,-0.007702,0.076648,RT
1,1/29/16 23:59,The #SuperBowl crew paints both end zones for ...,PatGeraghtyinIA,NaN,-0.003907,-0.011234,-0.007628,-0.076392,Th
2,1/29/16 23:59,RT @LevisStadium: Alright @Panthers. Your end ...,3timeallstar99,NaN,-0.003800,-0.006663,-0.006451,-0.064838,RT
3,1/29/16 23:59,RT @Panthers: Ready For Battle. \r\r#SB50 #Kee...,jessicavoncanno,NaN,-0.003231,-0.005244,-0.007628,0.203381,RT
4,1/29/16 23:59,RT @SapphireLV: #SuperBowl50 with @therealgian...,VEGASandTONIC,NaN,-0.002979,-0.008771,0.011353,-0.073192,RT
5,1/29/16 23:59,RT @CalFootball: Worth your 3 min. Tough not t...,slamdunk406,NaN,-0.003442,0.003865,-0.006525,-0.073192,RT
6,1/29/16 23:59,RT @PanthersCR: California Dreaming? 2 tickets...,Fiercely_,NaN,-0.000938,0.031871,0.002010,-0.063239,RT
7,1/29/16 23:59,Just got the pre-Super Bowl haircut. Feels lik...,NathanConley,NaN,-0.000556,0.013173,-0.000492,-0.076392,Ju
8,1/29/16 23:59,Who will win Super Bowl 50? _ُ� #SuperBowl #Su...,Juliann_Collins,NaN,-0.003876,-0.010242,-0.007702,-0.076392,Wh
9,1/29/16 23:59,RT mikepersinger: Thomas Davis in #SB50 with b...,ExploreTheMill,NaN,-0.001053,-0.008813,-0.001154,-0.076392,RT


###Create a list of top 50 users who may be considered the most influential in the domain you have chosen.
### We now calculate a score for each of tweet based on the weights calculated earlier

We then de-depulicate and gather the Top 50 Twitter handles based on this score

In [60]:
sbtweet['score'] = sbtweet['Follower']*w[0] + sbtweet['Listed']*w[1] + sbtweet['Retweet']*w[2]

result = sbtweet.sort(['score'], ascending= 0)['User Handle'].drop_duplicates().head(50)

In [61]:
result

1847         nflnetwork
707               Yahoo
4061            Broncos
1635         DrewFromTV
2906           RapSheet
1543           Panthers
1957           NFLonFOX
2054       YahooFinance
4360    BetTheSuperBowl
2669       AroundTheNFL
3759            GloZell
1203         jakemiller
3767              deray
1477    abc7newsBayArea
367             Digiday
207       BankofAmerica
528      USATODAYsports
1554           onlyinsf
2697       Ohio_Digital
1672         FuelOnline
4339         nbcbayarea
3235               KTVU
3580           NFLprguy
4585        11AliveNews
2174        theobserver
3708      peanuttillman
768                 7x7
4295        PostBroncos
86                wweek
4037          TroyRenck
1342        andytoronto
2841        BrewStudNo1
4046         LeoBurnett
917            SIRIUSXM
887            kmoxnews
2177       newsobserver
996         RonThornton
1360                UPI
4344      DonnaChaffins
1717         BackerClub
700         TAKNbySTORM
1654        sean

In [1]:
import pandas as pd
SBTweet=pd.read_csv('SuperBowl Tweet.csv')

In [2]:
SBTweet.head()

,Date,Tweet,User Handle,In Reply To,Follower,Following,Listed,Retweet
0,1/29/2016 23:59,RT @Panthers: Always With Us. #TEAM \r\r#SB50 ...,chanchanmans,NaN,167,137,1,861
1,1/29/2016 23:59,The #SuperBowl crew paints both end zones for ...,PatGeraghtyinIA,NaN,4,8,2,0
2,1/29/2016 23:59,RT @LevisStadium: Alright @Panthers. Your end ...,3timeallstar99,NaN,191,446,18,65
3,1/29/2016 23:59,RT @Panthers: Ready For Battle. \r\r#SB50 #Kee...,jessicavoncanno,NaN,1191,582,2,1574
4,1/29/2016 23:59,RT @SapphireLV: #SuperBowl50 with @therealgian...,VEGASandTONIC,NaN,1635,244,260,18


In [3]:
import json
import time
import re
def import_tweets(parameter1):
    
        match = re.match(r'RT\s@....+', parameter1)   
        if match:
            data = 'retweet'
        else:
            match = re.match(r'@....+', parameter1)
            if match:
                data = 'reply'
            else:
                match = re.match(r'....+@[^\s]\w*', parameter1)
                
                #match = re.match(r'....+@', parameter1)
                if match:
                    data = 'mention'
                else:
                    data = 'tweet'
        return data 

def sub(parameter1):
    parameter1 = re.sub('\\n\\n', '',parameter1)
    parameter1 = re.sub('\\r\\r', '',parameter1)
    parameter1 = re.sub('\\r', '',parameter1)
    parameter1 = re.sub('\\n', '',parameter1)
    return parameter1
    
import json
import time
import re
def Action(parameter1):
        parameter1= re.sub('[:]', '', parameter1)
        parameter1 = re.sub('\\n\\n ', '',parameter1)
        parameter1 = re.sub('\\r\\r ', '',parameter1)
        
        match = re.match(r'RT\s@....+', parameter1)   
        if match:
            data = re.split(r'RT\s@(\w*)\s|RT\s@(\w*):', parameter1)[1]
        else:
            match = re.match(r'@....+', parameter1)
            if match:
                data = re.split(r'@(\w*)\s|@(\w*)\w*|@(\w*)', parameter1)[1]
            else:
                match = re.match(r'....+@', parameter1)
                if match:
                    data = re.split(r'#*\w*\s*[\.*]*@(\w*)\s*\w*|@(\w*)\p{P}', parameter1)[1]
                    
                    #data = re.split(r'\.@(\w*)\s|@(\w*)$|@(\w*)\s|@(\w*):|@(\w*)\p{P}|@(\w*)', parameter1)[1]
                else:
                    data = 'No one'
        return data 
def Filler(x):
    if x=='Tweet':
        return SBTweet['User']

In [4]:
SBTweet['Type']=SBTweet['Tweet'].map(import_tweets)

In [5]:
SBTweet['Type'][7]=='tweet' # Check to see if tweet 8 is a tweet of type tweet

True

In [6]:
SBTweet['Action']=SBTweet['Tweet'].map(Action)

In [8]:
x=0
SBTweet['Col2']=SBTweet['Tweet']
while x<4741:
    if SBTweet['Type'][x]=='tweet':
        SBTweet['Col2'][x]=SBTweet['User Handle'][x]
        x=x+1
    else:
        SBTweet['Col2'][x]=SBTweet['Action'][x]
        x=x+1


C:\Users\Soundgarden\Anaconda\lib\site-packages\IPython\kernel\__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Soundgarden\Anaconda\lib\site-packages\IPython\kernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [9]:
SBTweet

,Date,Tweet,User Handle,In Reply To,Follower,Following,Listed,Retweet,Type,Action,Col2
0,1/29/2016 23:59,RT @Panthers: Always With Us. #TEAM \r\r#SB50 ...,chanchanmans,NaN,167,137,1,861,retweet,Panthers,Panthers
1,1/29/2016 23:59,The #SuperBowl crew paints both end zones for ...,PatGeraghtyinIA,NaN,4,8,2,0,mention,cbssports,cbssports
2,1/29/2016 23:59,RT @LevisStadium: Alright @Panthers. Your end ...,3timeallstar99,NaN,191,446,18,65,retweet,LevisStadium,LevisStadium
3,1/29/2016 23:59,RT @Panthers: Ready For Battle. \r\r#SB50 #Kee...,jessicavoncanno,NaN,1191,582,2,1574,retweet,Panthers,Panthers
4,1/29/2016 23:59,RT @SapphireLV: #SuperBowl50 with @therealgian...,VEGASandTONIC,NaN,1635,244,260,18,retweet,SapphireLV,SapphireLV
5,1/29/2016 23:59,RT @CalFootball: Worth your 3 min. Tough not t...,slamdunk406,NaN,821,1455,17,18,retweet,CalFootball,CalFootball
6,1/29/2016 23:59,RT @PanthersCR: California Dreaming? 2 tickets...,Fiercely_,NaN,5222,4139,133,74,retweet,PanthersCR,PanthersCR
7,1/29/2016 23:59,Just got the pre-Super Bowl haircut. Feels lik...,NathanConley,NaN,5894,2347,99,0,tweet,No one,NathanConley
8,1/29/2016 23:59,Who will win Super Bowl 50? _ُ� #SuperBowl #Su...,Juliann_Collins,NaN,58,103,1,0,tweet,No one,Juliann_Collins
9,1/29/2016 23:59,RT mikepersinger: Thomas Davis in #SB50 with b...,ExploreTheMill,NaN,5020,240,90,0,tweet,No one,ExploreTheMill


In [11]:
SB=SBTweet[['User Handle','Col2','Type']]
SB.columns=['Col1', 'Col2', 'Col3']



def at(x):
    return '@'+ str(x)

at('M')
SB['Col1']=SB['Col1'].map(at)
SB['Col2']=SB['Col2'].map(at)
SB

C:\Users\Soundgarden\Anaconda\lib\site-packages\IPython\kernel\__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Soundgarden\Anaconda\lib\site-packages\IPython\kernel\__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Col1,Col2,Col3
0,@chanchanmans,@Panthers,retweet
1,@PatGeraghtyinIA,@cbssports,mention
2,@3timeallstar99,@LevisStadium,retweet
3,@jessicavoncanno,@Panthers,retweet
4,@VEGASandTONIC,@SapphireLV,retweet
5,@slamdunk406,@CalFootball,retweet
6,@Fiercely_,@PanthersCR,retweet
7,@NathanConley,@NathanConley,tweet
8,@Juliann_Collins,@Juliann_Collins,tweet
9,@ExploreTheMill,@ExploreTheMill,tweet


In [12]:
SB.to_csv('SMAAssignment2PartB.csv',index=False)